In [1]:
import quartz
from QGNN import QGNN, train_supervised
import torch
import time

Using backend: pytorch
[17:02:12] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/lzk/anaconda3/envs/quantum/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory


In epoch 0, loss: 2.485, train acc: 0.917, val acc: 1.000 (best 1.000), test acc: 0.667 (best 0.667)
In epoch 5, loss: 0.163, train acc: 0.917, val acc: 0.333 (best 1.000), test acc: 1.000 (best 0.667)
In epoch 10, loss: 0.023, train acc: 1.000, val acc: 0.667 (best 1.000), test acc: 1.000 (best 0.667)
In epoch 15, loss: 0.020, train acc: 1.000, val acc: 1.000 (best 1.000), test acc: 1.000 (best 0.667)


In [2]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')
parser = quartz.PyQASMParser(context=quartz_context)
my_dag = parser.load_qasm(filename="../circuit/nam-circuits/qasm_files/tof_4_before.qasm")
my_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)
my_graph_dgl = my_graph.to_dgl_graph()

In [3]:
# start = time.time()
# for node in my_graph.all_nodes():
#     print(my_graph.available_xfers(context=quartz_context, node=node, output_format="int"))
# f"{time.time() - start:.4f}s"

In [4]:
start = time.time()
appliable_xfer_matrix = my_graph.get_available_xfers_matrix(context=quartz_context)
print(f"{time.time() - start:.4f}s")
my_graph_dgl.ndata['label'] = torch.tensor(appliable_xfer_matrix,dtype=torch.float)

93.1589s


In [5]:
num_data = my_graph.num_nodes
val_rate = 0.15
test_rate = 0.15
val_num = int(num_data * val_rate)
test_num = int(num_data * test_rate)
train_num = num_data - val_num - test_num
train_mask = [1] * train_num + [0] * val_num + [0] * test_num
val_mask = [0] * train_num + [1] * val_num + [0] * test_num
test_mask = [0] * train_num + [0] * val_num + [1] * test_num

my_graph_dgl.ndata['train_mask'] = torch.tensor(train_mask,dtype=torch.bool) 
my_graph_dgl.ndata['val_mask'] = torch.tensor(val_mask,dtype=torch.bool) 
my_graph_dgl.ndata['test_mask'] = torch.tensor(test_mask,dtype=torch.bool) 
model = QGNN(26, 16, quartz_context.num_xfers, 16)

In [6]:
def test(*, filename):
    test_dag = parser.load_qasm(filename=filename)
    test_graph = quartz.PyGraph(context=quartz_context, dag=test_dag)
    test_graph_dgl = test_graph.to_dgl_graph()
    appliable_xfer_matrix = test_graph.get_available_xfers_matrix(context=quartz_context)
    test_graph_dgl.ndata['label'] = torch.tensor(appliable_xfer_matrix,dtype=torch.float)
    labels = test_graph_dgl.ndata['label']

    with torch.no_grad():
        logits = model(test_graph_dgl)
        pred = logits > 0.5
        test_acc = (pred == labels).float().mean()
        print(f"test_acc: {test_acc:.6f}")

In [7]:
test(filename="../circuit/nam-circuits/qasm_files/tof_3_before.qasm")

test_acc: 0.933595


In [8]:
test(filename="../circuit/nam-circuits/qasm_files/tof_5_before.qasm")

test_acc: 0.934449


In [9]:
train_supervised(my_graph_dgl, model)

In epoch 0, loss: 0.111, train acc: 0.937, val acc: 0.934 (best 0.934), test acc: 0.922 (best 0.922)
In epoch 5, loss: 0.005, train acc: 0.999, val acc: 0.999 (best 0.999), test acc: 0.999 (best 0.999)
In epoch 10, loss: 0.002, train acc: 0.999, val acc: 0.999 (best 0.999), test acc: 0.999 (best 0.999)
In epoch 15, loss: 0.001, train acc: 0.999, val acc: 0.999 (best 0.999), test acc: 0.999 (best 0.999)


In [10]:
test(filename="../circuit/nam-circuits/qasm_files/tof_3_before.qasm")

test_acc: 0.998881


In [11]:
test(filename="../circuit/nam-circuits/qasm_files/tof_5_before.qasm")

test_acc: 0.998865


In [12]:
# test(filename="../circuit/nam-circuits/qasm_files/tof_10_before.qasm")